## **01 Web Scrape Reviews**
This script takes a list of breweries and scrapes the corresponding Tripadvisor review data. The brewery list is sourced from Open Brewery DB.

### **Notebook Objectives**
1. Import a CSV into pandas and extract a list of breweries to scrape
2. For each brewery in the list, use the requests package to query DuckDuckGo and find the corresponding Tripadvisor page (if exists)
3. Scrape and save the html page using dill in case the scraped content is needed at a later date
4. Save the scraped review data as json files for later processing in the next notebook
5. Demonstrate storing the scraped data in a SQL database as an alternative to json files

In [3]:
import os
from pathlib import Path
from dotenv import dotenv_values
import requests
from bs4 import BeautifulSoup
import time
import random
import dill
import pandas as pd
import re
import json
import logging
from sqlalchemy import create_engine
from helper import parse_soup

logging.basicConfig(filename='scrape.log', filemode='w', 
        level=logging.DEBUG, force=True,
        format='%(asctime)s %(levelname)s:%(message)s')

config = dotenv_values(dotenv_path=Path('../.env'))

In [4]:
# s = requests.Session()

def get_soup(url):
    headers = {'User-Agent': config['USER_AGENT']}
    response = requests.get(url, headers=headers)
    # response = s.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup, response

def get_brew_id(id, url, review_type):
    # check tripadviser page type
    if review_type == 'Attraction':
        brew_id = re.sub('https://www.tripadvisor.+/Attraction_Review', '-Attraction', url)
    elif review_type == 'Restaurant':
        brew_id = re.sub('https://www.tripadvisor.+/Restaurant_Review', '-Restaurant', url)
    else:
        raise TypeError('Unsupported review type')
    brew_id = id + brew_id.strip('.html')
    return brew_id

def save_page(response, page, id):
    name = '../assets/html/' + id + page + '.dill'
    with open(name, 'wb') as f:
        dill.dump(response, f)
    return

def save_json(id, page, contents):
    with open(f'../assets/json/{id}{page}.json', 'w') as f:
        json.dump(contents, f)
    return

def increment_url(url, page):
    url = url.replace('-Reviews', '-Reviews' + page)
    return url

def get_review_type(url):
    if 'Attraction_Review' in url:
        # scrape up to 30 reviews
        pages = ['', '-or10', '-or20']
        return 'Attraction', pages
    elif 'Restaurant_Review' in url:
        # scrape up to 45 reviews since the lengths are truncated in Restaurant case
        pages = ['', '-or15', '-or30']
        return 'Restaurant', pages
    else:
        logging.warning('URL does not contain proper format')
        return None, None

def scrape(id, url_base):
    """
    Inputs: base url (str), brewery name (str)
    """
    # determine if tripadvisor url classifies it as attraction or restaurant
    review_type, pages = get_review_type(url_base)
    # skip if url does not follow format
    if review_type is not None:
        # get unique brewery identifier
        brew_id = get_brew_id(id, url_base, review_type)
        # loop through a couple pages of reviews
        review_list = []
        for page in pages:
            url = increment_url(url_base, page)
            logging.info(f'Scraping: {url}')
            soup, response = get_soup(url)
            save_page(response, page, brew_id)
            review_data = parse_soup(soup, id, review_type)
            if review_data:
                save_json(id, page, review_data)
                logging.debug('Page scraped!')
                review_list.append(review_data)
            time.sleep(random.randint(3,5))
    return

def get_url_base(name, state, city, verbose=False):
    """
    Inputs: brewery name, state
    """
    base = 'https://duckduckgo.com/html/?q='
    name = name.replace(' ', '+')
    state = state.replace(' ', '+')
    city = city.replace(' ', '+')
    url = f'{base}+tripadvisor+{name}+{state}+{city}'
    if verbose:
        logging.debug(url)
    soup, response = get_soup(url)
    links = soup.find_all("a", class_="result__url", href=True)
    return links[0]['href']

In [6]:
# Test HTML requests is working
df = pd.read_csv(Path('../assets/breweries_clean_address.csv'))
states = 'Massachusetts'
df_states = df.query('state in @states')
breweries_subset = df_states[['obdb_id', 'name', 'state', 'city', 'street', 'longitude', 'latitude', 'website_url']]
print(f'Brewery df shape: {breweries_subset.shape}')

# Test search for Tripadvisor link
test_brew = breweries_subset.iloc[21]
link = get_url_base(test_brew['name'], test_brew['state'], 
    test_brew['city'], verbose=True)
print(test_brew['name'], link)

# Test scrape
time.sleep(1)
soup, response = get_soup(link)
logging.debug(soup.text)

Brewery df shape: (163, 8)
Bent Water Brewing Company https://www.tripadvisor.com/Restaurant_Review-g41651-d10214726-Reviews-Bent_Water_Brewing_Company-Lynn_Massachusetts.html


In [7]:
# soup.find('div', class_='_d MJ').text
# soup.find('div', class_='wgNTK').find('span').text
soup.find('span', class_='yEWoV').text

'180 Commercial St Unit 18, Lynn, MA 01905-2910'

In [8]:
def get_subset(state):
    '''Get list of breweries for a single state'''
    df = pd.read_csv(Path('../assets/breweries_clean_address.csv'))
    df_subset = df.query('state in @state').reset_index()
    columns = ['obdb_id', 'name', 'state', 'city', 'street', 'longitude', 'latitude', 'website_url']
    df_subset = df_subset[columns]
    return df_subset

def scrape_breweries(state, start=0, end=-1):
    '''Scrape brewery reviews for datatframe input'''
    df_subset = get_subset(state)
    count = df_subset.shape[0]-1
    # Scrape reviews
    for index, brewery in df_subset[start:end].iterrows():
        id = brewery['obdb_id']
        name = brewery['name']
        city = brewery['city']
        name_token = re.findall(r'^[A-Za-z\d]+', name)[0]
        state = brewery['state']
        url_base = get_url_base(name, state, city)
        logging.info(f'Brewery {index} out of {count}')
        logging.info(url_base)
        if 'tripadvisor.' in url_base and name_token in url_base:
            scrape(id, url_base)
            # save_json(id, contents)
        elif 'tripadvisor.' in url_base:
            name_token = name_token.strip('s')
            if 'tripadvisor.' in url_base and name_token in url_base:
                scrape(id, url_base)
                # save_json(id, contents)
        else:
            logging.info(f'No trip advisor result for {name}')
    return df_subset

In [9]:
# Get size of brewery subset
state = 'Massachusetts'
df_subset = get_subset(state)
print(df_subset.shape)
print(df_subset.iloc[22])

(163, 8)
obdb_id        berkley-beer-company-berkley
name                   Berkley Beer Company
state                         Massachusetts
city                                Berkley
street                                  NaN
longitude                               NaN
latitude                                NaN
website_url      http://www.berkleybeer.com
Name: 22, dtype: object


In [25]:
# Scrape selected state
df_scrape = scrape_breweries(state, start=25, end=27)
df_scrape.shape

(163, 8)

In [63]:
# TODO Practice storing the scraped data in a SQL database
from sqlalchemy import Column, Date, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker

engine = create_engine("sqlite:///../assets/foo.db")
Base = declarative_base()

class City(Base):

    __tablename__ = "cities"

    id = Column(Integer, primary_key=True)
    name = Column(String)  

    # def __init__(self, name):
    #     self.name = name    

Base.metadata.create_all(engine)

In [64]:
# Set up the Session
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [66]:
print(f'Count: {session.query(City).count()}')

names = ['A', 'B', 'C']
ids = ['1', '2', '3']
for name, id in zip(names, ids):
    if session.query(City).filter(City.id==id).first() is None:
        new_city = City(name=name, id=id)
        session.add(new_city)

# Write to DB
session.commit()

print(f'New Count: {session.query(City).count()}')

Count: 3
New Count: 3


In [67]:
# Inspect table
for city in session.query(City).limit(10):
    print(city.id, city.name)

1 A
2 B
3 C
